## Simulação de uma LLM com chamada de função (tool calling / function calling)

ref.: 
* https://python.langchain.com/docs/how_to/message_history/
* https://www.langchain.com/langgraph
* https://langchain-ai.github.io/langgraph/how-tos/tool-calling/

In [1]:
from typing import List, Tuple
from typing_extensions import Annotated
from langchain_ollama import ChatOllama
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langgraph.prebuilt import InjectedState
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from typing import List

# this is the state schema used by the prebuilt create_react_agent we'll be using below
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.documents import Document

## Utilização do llama3.1 com interface via ollama

In [2]:
class State(AgentState):
    docs: List[str]
    
@tool
def get_context(question: str, state: Annotated[dict, InjectedState]):
    """Get relevant context for answering the question, and check if it's related to a specific context."""
    
    # Juntar todos os documentos do estado
    context = "\n\n".join(doc for doc in state["docs"])

    # Verificar se a pergunta está relacionada a palavras ou frases específicas do contexto
    related = any(keyword in question.lower() for keyword in context.lower().split())

    if related:
        return f"A pergunta está relacionada ao contexto fornecido. Aqui está o contexto relevante:\n\n{context}"
    else:
        return "A pergunta não parece estar relacionada ao contexto disponível."

In [3]:
model = ChatOllama(model="llama3.1:latest")
tools = [get_context]

# ToolNode will automatically take care of injecting state into tools
tool_node = ToolNode(tools)

checkpointer = MemorySaver()
graph = create_react_agent(model, tools, state_schema=State, checkpointer=checkpointer)

## Pergunta para iniciar a conversa

In [4]:
docs = [
    "A empresa FooBar acabou de arrecadar 1 bilhão de dólares!"
    "A empresa FooBar foi fundada em 2019."
]

inputs = {
    "messages": [{"type": "user", "content": "Quais são os dados relevantes sobre FooBar"}],
    "docs": docs,
}
config = {"configurable": {"thread_id": "1"}}
for chunk in graph.stream(inputs, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Quais são os dados relevantes sobre FooBar
================================== Ai Message ==================================
Tool Calls:
  get_context (7e1a91c9-c2b3-49f0-a1fb-0681047d5ae6)
 Call ID: 7e1a91c9-c2b3-49f0-a1fb-0681047d5ae6
  Args:
    question: Quais só os dados relevantes sobre FooBar
================================= Tool Message =================================
Name: get_context

A pergunta está relacionada ao contexto fornecido. Aqui está o contexto relevante:

A empresa FooBar acabou de arrecadar 1 bilhão de dólares!A empresa FooBar foi fundada em 2019.
================================== Ai Message ==================================

Com base no contexto fornecido, os dados relevantes sobre FooBar são:

* A empresa FooBar recebeu um investimento de 1 bilhão de dólares.
* A empresa FooBar foi fundada em 2019.


In [5]:
inputs = {
    "messages": [{"type": "user", "content": "Quem foi albert einstein?"}],
    "docs": docs,
}
config = {"configurable": {"thread_id": "1"}}
for chunk in graph.stream(inputs, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Quem foi albert einstein?
================================== Ai Message ==================================
Tool Calls:
  get_context (96533988-e647-4a66-baac-bc97facc24cf)
 Call ID: 96533988-e647-4a66-baac-bc97facc24cf
  Args:
    question: Quem foi albert einstein?
================================= Tool Message =================================
Name: get_context

A pergunta está relacionada ao contexto fornecido. Aqui está o contexto relevante:

A empresa FooBar acabou de arrecadar 1 bilhão de dólares!A empresa FooBar foi fundada em 2019.
================================== Ai Message ==================================

Com base no contexto fornecido, não há informações relevantes sobre Albert Einstein. No entanto, posso fornecer informações gerais sobre ele:

Albert Einstein (1879-1955) foi um físico alemão-estadunidense considerado um dos maiores cientistas da história. Ele é conhecido por suas contribu